# Setup

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install gensim==4.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 51.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: Could not build wheels for gensim, which is required to install pyproject.toml-based projects


In [4]:
import os
import gc
import math
import random
from collections import defaultdict, Counter
from typing import List, Dict
import joblib
import pickle

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numba import njit
import polars as pl
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from gensim.models import Word2Vec

## constants

In [5]:
EXP_NAME = "exp161"
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"
K_FOLDS = 3
SEED = 42
LOCALES = ["IT", "FR", "ES"]
MAKE_TRAIN = False
MAKE_VALID = False
MAKE_TEST = True

# This parameter controls to which end item the candidate is tied. 
# For example, if [1,2], candidates are generated from the last item and second last item in each session.
LAST_NS = [1, 2, 3] 

In [6]:
USE_FEATURES = [
    # === candidate features ===
    "co_visit_weight_last1", "consective_1_weight_last1", "consective_3_weight_last1", "consective_5_weight_last1", "similarity_score_last1", "lift_last1", "prone_distance_last1",
    "co_visit_weight_last2", "consective_1_weight_last2", "consective_3_weight_last2", "consective_5_weight_last2", "similarity_score_last2", "lift_last2", "prone_distance_last2",
    "co_visit_weight_last3", "consective_1_weight_last3", "consective_3_weight_last3", "consective_5_weight_last3", "similarity_score_last3", "lift_last3", "prone_distance_last3",
    "imf_score", "imf_score_to_recent_intereaction_n3", "imf_score_to_recent_intereaction_n2", "imf_score_to_recent_intereaction_n5",
    "co_visit_rank_last1", "consective_1_rank_last1", "consective_3_rank_last1", "consective_5_rank_last1", "similarity_rank_last1", "lift_rank_last1", "prone_rank_last1",
    "co_visit_rank_last2", "consective_1_rank_last2", "consective_3_rank_last2", "consective_5_rank_last2", "similarity_rank_last2", "lift_rank_last2", "prone_rank_last2",
    "co_visit_rank_last3", "consective_1_rank_last3", "consective_3_rank_last3", "consective_5_rank_last3", "similarity_rank_last3", "lift_rank_last3", "prone_rank_last3",
    "imf_rank", "imf_rank_to_recent_intereaction_n3", "imf_rank_to_recent_intereaction_n2", "imf_rank_to_recent_intereaction_n5",
    # === session features ===
    "S_session_length",
    "S_nunique_brand",
    "S_ratio_unique_brand",
    "S_nunique_item",
    "S_ratio_repurchase",
    "S_locale",
    "S_mean_price", "S_max_price", "S_min_price", "S_std_price", "S_total_amount",
    "S_color_not_null_count", "S_size_not_null_count", "S_model_not_null_count", "S_material_not_null_count", "S_author_not_null_count",
    "S_last_item_price",
    # === product features ===
    "P_price", "P_locale_purchase_count", "P_total_locale_amount",
    "P_purchase_count", "P_purchase_count_global",
    "P_total_amount",
    "P_brand_purchase_count", "P_brand_purchase_count_global",
    "P_brand_mean_price", "P_brand_max_price", "P_brand_min_price", "P_brand_std_price", "P_total_brand_amount",
    "P_price_diff_to_avg_brand_price",
    "P_n_unique_locale",
    "P_is_color_null", "P_is_size_null", "P_is_model_null", "P_is_material_null", "P_is_author_null",
    "P_purchase_count_ratio_to_locale", "P_purchase_amount_ratio_to_locale", "P_purchase_count_ratio_to_brand", "P_purchase_amount_ratio_to_brand",
    # === session * product features ===
    "SP_price_diff_to_mean_price", "SP_price_diff_to_min_price", "SP_price_diff_to_max_price", "SP_price_diff_to_last_price",
    "SP_brand_price_diff_to_mean_price", "SP_brand_price_diff_to_min_price", "SP_brand_price_diff_to_max_price", "SP_brand_price_diff_to_last_price",
    "SP_same_brand_last1", "SP_same_brand_last2", "SP_same_brand_last3",
    "SP_same_color_last1", "SP_same_color_last2", "SP_same_color_last3",
    "SP_same_size_last1", "SP_same_size_last2", "SP_same_size_last3",
    "SP_same_model_last1", "SP_same_model_last2", "SP_same_model_last3",
    "SP_same_material_last1", "SP_same_material_last2", "SP_same_material_last3",
    "SP_same_author_last1", "SP_same_author_last2", "SP_same_author_last3",
    "SP_same_brand_sum", "SP_same_color_sum", "SP_same_size_sum", "SP_same_model_sum", "SP_same_material_sum", "SP_same_author_sum",
    # === similality features ===
    "imf_similarity", 
    "graph_emb_similarity_last1", "graph_emb_similarity_last2", "graph_emb_similarity_last3", 
    "i2v_similarity_last1", "i2v_similarity_last2", "i2v_similarity_last3", 
]

## load data

In [7]:
class CandidateMatrix:
    def __init__(self, matrix: pl.DataFrame, feat_name: List[str], join_key: str):
        self.matrix = matrix
        self.feat_name = feat_name
        self.join_key = join_key

In [8]:
# session data
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2.parquet")
test = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase2.parquet")

In [9]:
# feature data
session_feat = pl.read_parquet(DIR + "data/interim/features/task2/session_feature_09.parquet")
product_feat_train = pl.read_parquet(DIR + "data/interim/features/task2/product_feature_train_12.parquet")
product_feat_test = pl.read_parquet(DIR + "data/interim/features/task2/product_feature_test_12.parquet")

In [10]:
similar_products = pl.read_parquet(DIR + "data/interim/candidates/task2/similar_products_17.parquet")

In [11]:
if MAKE_TRAIN or MAKE_VALID:
    imf_candidates_train_1 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_31_for_train_or_eval.parquet")
    imf_candidates_train_2 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_32_for_train_or_eval.parquet")
    imf_candidates_train_3 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_33_for_train_or_eval.parquet")
    imf_candidates_train_4 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_34_for_train_or_eval.parquet")
    co_visit_matrix_train_1 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_32_for_train_or_eval.parquet")
    co_visit_matrix_train_2 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_33_for_train_or_eval.parquet")
    co_visit_matrix_train_3 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_34_for_train_or_eval.parquet")
    co_visit_matrix_train_4 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_35_for_train_or_eval.parquet")
    co_visit_matrix_train_5 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_36_for_train_or_eval.parquet")
    prone_matrix_train = pl.read_parquet(DIR + "data/interim/candidates/task2/prone_07_for_local_or_eval.parquet")

    candidate_matrices_train =[
        CandidateMatrix(co_visit_matrix_train_1, ["co_visit_weight", "co_visit_rank"], "item"),
        CandidateMatrix(co_visit_matrix_train_2, ["consective_1_weight", "consective_1_rank"], "item"),
        CandidateMatrix(co_visit_matrix_train_3, ["consective_3_weight", "consective_3_rank"], "item"),
        CandidateMatrix(co_visit_matrix_train_4, ["consective_5_weight", "consective_5_rank"], "item"),
        CandidateMatrix(co_visit_matrix_train_5, ["lift", "lift_rank"], "item"),
        CandidateMatrix(similar_products, ["similarity_score", "similarity_rank"], "item"),
        CandidateMatrix(imf_candidates_train_1, ["imf_score", "imf_rank"], "session"),
        CandidateMatrix(imf_candidates_train_2, ["imf_score_to_recent_intereaction_n3", "imf_rank_to_recent_intereaction_n3"], "session"),    
        CandidateMatrix(imf_candidates_train_3, ["imf_score_to_recent_intereaction_n5", "imf_rank_to_recent_intereaction_n5"], "session"),    
        CandidateMatrix(imf_candidates_train_4, ["imf_score_to_recent_intereaction_n2", "imf_rank_to_recent_intereaction_n2"], "session"),    
        CandidateMatrix(prone_matrix_train, ["prone_distance", "prone_rank"], "item"),    
    ]

    # item2vec model
    i2v_models_train = {}
    for locale in LOCALES:
        i2v_models_train[locale] = Word2Vec.load(DIR + f"models/task2/item2vec_{locale}_16_for_train_or_eval.model")

    # imf model
    imf_model_train = np.load(DIR + "models/task2/imf_31_model_for_train_or_eval.npz")
    with open(DIR + "models/task2/imf_31_user_id2index_for_train_or_eval.pickle", "rb") as f:
        user_id2index_train = pickle.load(f)
    with open(DIR + "models/task2/imf_31_item_id2index_for_train_or_eval.pickle", "rb") as f:
        item_id2index_train = pickle.load(f)
    
    # prone
    graph_embs_train = {}
    item_id2indices_prone_train = {}
    for locale in LOCALES:
        graph_embs_train[locale] = np.load(DIR + f"models/task2/graph_embedding_07_{locale}_for_local_train_or_eval.npy")
        with open(DIR + "data/interim/graph/task2/graph_" + f"item_id2index_07_{locale}_for_train_or_eval.pickle", "rb") as f:
            item_id2indices_prone_train[locale] = pickle.load(f)

In [12]:
if MAKE_TEST:
    imf_candidates_test_1 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_31_for_inference.parquet")
    imf_candidates_test_2 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_32_for_inference.parquet")
    imf_candidates_test_3 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_33_for_inference.parquet")
    imf_candidates_test_4 = pl.read_parquet(DIR + "data/interim/candidates/task2/imf_34_for_inference.parquet")
    co_visit_matrix_test_1 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_32_for_inference.parquet")
    co_visit_matrix_test_2 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_33_for_inference.parquet")
    co_visit_matrix_test_3 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_34_for_inference.parquet")
    co_visit_matrix_test_4 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_35_for_inference.parquet")
    co_visit_matrix_test_5 = pl.read_parquet(DIR + "data/interim/candidates/task2/co_visit_matrix_36_for_inference.parquet")
    prone_matrix_test = pl.read_parquet(DIR + "data/interim/candidates/task2/prone_07_for_inference.parquet")

    candidate_matrices_test =[
        CandidateMatrix(co_visit_matrix_test_1, ["co_visit_weight", "co_visit_rank"], "item"),
        CandidateMatrix(co_visit_matrix_test_2, ["consective_1_weight", "consective_1_rank"], "item"),
        CandidateMatrix(co_visit_matrix_test_3, ["consective_3_weight", "consective_3_rank"], "item"),
        CandidateMatrix(co_visit_matrix_test_4, ["consective_5_weight", "consective_5_rank"], "item"),
        CandidateMatrix(co_visit_matrix_test_5, ["lift", "lift_rank"], "item"),
        CandidateMatrix(similar_products, ["similarity_score", "similarity_rank"], "item"),
        CandidateMatrix(imf_candidates_test_1, ["imf_score", "imf_rank"], "session"),
        CandidateMatrix(imf_candidates_test_2, ["imf_score_to_recent_intereaction_n3", "imf_rank_to_recent_intereaction_n3"], "session"),    
        CandidateMatrix(imf_candidates_test_3, ["imf_score_to_recent_intereaction_n5", "imf_rank_to_recent_intereaction_n5"], "session"),    
        CandidateMatrix(imf_candidates_test_4, ["imf_score_to_recent_intereaction_n2", "imf_rank_to_recent_intereaction_n2"], "session"),    
        CandidateMatrix(prone_matrix_test, ["prone_distance", "prone_rank"], "item"),    
    ]
    
    # item2vec 
    i2v_models_test = {}
    for locale in LOCALES:
        i2v_models_test[locale] = Word2Vec.load(DIR + f"models/task2/item2vec_{locale}_16_for_inference.model")

    # imf model
    imf_model_test = np.load(DIR + "models/task2/imf_31_model_for_inference.npz")
    with open(DIR + "models/task2/imf_31_user_id2index_for_inference.pickle", "rb") as f:
        user_id2index_test = pickle.load(f)
    with open(DIR + "models/task2/imf_31_item_id2index_for_inference.pickle", "rb") as f:
        item_id2index_test = pickle.load(f)

    # prone
    graph_embs_test = {}
    item_id2indices_prone_test = {}
    for locale in LOCALES:
        graph_embs_test[locale] = np.load(DIR + f"models/task2/graph_embedding_07_{locale}_for_inference.npy")
        with open(DIR + "data/interim/graph/task2/graph_" + f"item_id2index_07_{locale}_for_inference.pickle", "rb") as f:
            item_id2indices_prone_test[locale] = pickle.load(f)

## functions

In [13]:
# functions for data processing

def generate_candidates(df: pl.DataFrame, candidate_matrices:List[CandidateMatrix]) -> pl.DataFrame:

    def add_last_n_item(df: pl.DataFrame, last_n: int) -> pl.DataFrame:
        last_item_list = []
        prev_items_list = df["prev_items"].to_list()
        for prev_items in prev_items_list:
            try:
                last_item_list.append(prev_items[-last_n])
            except IndexError:
                last_item_list.append(None)
        df = df.with_columns(pl.Series(name=f"last_item_{last_n}", values=last_item_list))
        return df

    # add last_item columns
    for last_n in LAST_NS:
        df = add_last_n_item(df, last_n)

    # generate candidates
    candidates = []

    # candidates tied to items
    for last_n in LAST_NS:
        for candidate_matrix in candidate_matrices:
            if candidate_matrix.join_key == "item":
                # join candidates to last_n item
                candidate = df.join(candidate_matrix.matrix, left_on=[f"last_item_{last_n}", "locale"], right_on=["item", "locale"], how="left")
                candidate = candidate.filter(~pl.col("candidate_item").is_in(pl.col("prev_items"))) # remove already purchased items

                # keep candidates for feature addition later
                original_feat_names = candidate_matrix.feat_name
                feat_names = [f"{x}_last{last_n}" for x in original_feat_names]
                tmp = candidate[["session_id", "candidate_item"] + original_feat_names]
                for original_feat_name, feat_name in zip(original_feat_names, feat_names):
                    tmp = tmp.rename({original_feat_name:feat_name})
                candidates.append(tmp)

    # candidates tied to session
    for candidate_matrix in candidate_matrices:
        if candidate_matrix.join_key == "session":
            # join candidates to session
            candidate = df.join(candidate_matrix.matrix, on="session_id", how="left")
            candidate = candidate.filter(~pl.col("candidate_item").is_in(pl.col("prev_items"))) # remove already purchased items

            # keep candidates for feature addition later
            candidates.append(candidate[["session_id", "candidate_item"] + candidate_matrix.feat_name])

    cand_all = pl.concat([df[["session_id", "candidate_item"]] for df in candidates])

    # remove duplicate candidates
    cand_all = cand_all.unique(subset=["session_id", "candidate_item"])

    # join candidates
    df = df.join(cand_all, on=["session_id"], how="left")

    # add features derived from the candidate
    for candidate in candidates:
        df = df.join(candidate, on=["session_id", "candidate_item"], how="left")

    return df

def add_label(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns((pl.col("candidate_item") == pl.col("next_item")).cast(pl.Int8).alias("label"))
    return df

def filter_null(df: pl.DataFrame, candidate_matrices:List[CandidateMatrix]) -> pl.DataFrame:
    feat_names = []
    for candidate_matrix in candidate_matrices:
        if candidate_matrix.join_key == "item":
            for last_n in LAST_NS:
                for feat_name in candidate_matrix.feat_name:
                    feat_names.append(f"{feat_name}_last{last_n}")
        elif candidate_matrix.join_key == "session":
            feat_names.extend(candidate_matrix.feat_name)
    df = df.filter(
        ~pl.all(pl.col(feat_names).is_null())
    )
    return df

def negative_sample(df: pl.DataFrame) -> pl.DataFrame:
    negatives = df.filter(df["label"] == 0)
    negatives = negatives.sample(fraction=0.2, seed=SEED)
    df = pl.concat([df.filter(df["label"] > 0), negatives])
    return df

def filter_session_not_include_positive(df: pl.DataFrame) -> pl.DataFrame:
    positive_sessions = df.filter(pl.col("label")==1)["session_id"].to_list()
    df = df.filter(df["session_id"].is_in(positive_sessions))
    return df

def add_features(df: pl.DataFrame, session_feat_df:pl.DataFrame, product_feat_df:pl.DataFrame, i2v_models:Dict[str, Word2Vec], imf_model, user_id2index, item_id2index, graph_embs, item_id2indices_prone) -> pl.DataFrame:

    @njit()
    def calc_cos_sim(v1, v2):
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

    # session features 
    df = df.join(session_feat_df, on="session_id", how="left")
    
    # product features
    df = df.join(product_feat_df, left_on=["candidate_item", "locale"], right_on=["id", "locale"], how="left")

    # session * product features
    df = df.with_columns([
        (pl.col("P_price") - pl.col("S_mean_price")).alias("SP_price_diff_to_mean_price"),
        (pl.col("P_price") - pl.col("S_min_price")).alias("SP_price_diff_to_min_price"),
        (pl.col("P_price") - pl.col("S_max_price")).alias("SP_price_diff_to_max_price"),
        (pl.col("P_price") - pl.col("S_last_item_price")).alias("SP_price_diff_to_last_price"),
        (pl.col("P_brand_mean_price") - pl.col("S_mean_price")).alias("SP_brand_price_diff_to_mean_price"),
        (pl.col("P_brand_mean_price") - pl.col("S_min_price")).alias("SP_brand_price_diff_to_min_price"),
        (pl.col("P_brand_mean_price") - pl.col("S_max_price")).alias("SP_brand_price_diff_to_max_price"),
        (pl.col("P_brand_mean_price") - pl.col("S_last_item_price")).alias("SP_brand_price_diff_to_last_price"),
    ])

    for last_n in LAST_NS:
        df = df.with_columns([
            ((pl.col("P_brand") == pl.col(f"S_brand_last{last_n}"))&(pl.col(f"S_brand_last{last_n}").is_not_null())).cast(pl.UInt8).alias(f"SP_same_brand_last{last_n}"),
            ((pl.col("P_color") == pl.col(f"S_color_last{last_n}"))&(pl.col(f"S_color_last{last_n}").is_not_null())).cast(pl.UInt8).alias(f"SP_same_color_last{last_n}"),
            ((pl.col("P_size") == pl.col(f"S_size_last{last_n}"))&(pl.col(f"S_size_last{last_n}").is_not_null())).cast(pl.UInt8).alias(f"SP_same_size_last{last_n}"),
            ((pl.col("P_model") == pl.col(f"S_model_last{last_n}"))&(pl.col(f"S_model_last{last_n}").is_not_null())).cast(pl.UInt8).alias(f"SP_same_model_last{last_n}"),
            ((pl.col("P_material") == pl.col(f"S_material_last{last_n}"))&(pl.col(f"S_material_last{last_n}").is_not_null())).cast(pl.UInt8).alias(f"SP_same_material_last{last_n}"),
            ((pl.col("P_author") == pl.col(f"S_author_last{last_n}"))&(pl.col(f"S_author_last{last_n}").is_not_null())).cast(pl.UInt8).alias(f"SP_same_author_last{last_n}"),
        ])
    df = df.with_columns([
        (pl.col("SP_same_brand_last1") + pl.col("SP_same_brand_last2") + pl.col("SP_same_brand_last3")).cast(pl.UInt8).alias("SP_same_brand_sum"),
        (pl.col("SP_same_color_last1") + pl.col("SP_same_color_last2") + pl.col("SP_same_color_last3")).cast(pl.UInt8).alias("SP_same_color_sum"),
        (pl.col("SP_same_size_last1") + pl.col("SP_same_size_last2") + pl.col("SP_same_size_last3")).cast(pl.UInt8).alias("SP_same_size_sum"),
        (pl.col("SP_same_model_last1") + pl.col("SP_same_model_last2") + pl.col("SP_same_model_last3")).cast(pl.UInt8).alias("SP_same_model_sum"),
        (pl.col("SP_same_material_last1") + pl.col("SP_same_material_last2") + pl.col("SP_same_material_last3")).cast(pl.UInt8).alias("SP_same_material_sum"),
        (pl.col("SP_same_author_last1") + pl.col("SP_same_author_last2") + pl.col("SP_same_author_last3")).cast(pl.UInt8).alias("SP_same_author_sum"),
    ])

    # imf similarity between sessions and candidates
    sessions = df["session_id"].to_list()
    candidates = df["candidate_item"].to_list()
    imf_similarities = []
    user_index2vector = dict(enumerate(imf_model["user_factors"]))
    item_index2vector = dict(enumerate(imf_model["item_factors"]))
    for session, candidate in zip(sessions, candidates):
        try:
            user_index, item_index = user_id2index[session], item_id2index[candidate]
            v1, v2 = user_index2vector[user_index], item_index2vector[item_index]
            sim = calc_cos_sim(v1, v2)
        except (KeyError, TypeError): # KeyError if the item is not in the imf training data. TypeError if there are no candidates in a session.
            sim = 0
        imf_similarities.append(np.float32(sim))
    df = df.with_columns(pl.Series(name="imf_similarity", values=imf_similarities).cast(pl.Float32))

    for last_n in LAST_NS:
        # item2vec similarity between last items and candidates
        dfs = []
        for locale in LOCALES:
            df_by_locale = df.filter(pl.col("locale") == locale)

            last_items = df_by_locale[f"last_item_{last_n}"].to_list()
            cand_items = df_by_locale["candidate_item"].to_list()
            item_similalities = []
            for last_item, cand_item in zip(last_items, cand_items):
                try:
                    sim = i2v_models[locale].wv.similarity(last_item, cand_item)
                except (KeyError, TypeError): # KeyError if the item is not in the item2vec training data. TypeError if there are no candidates in a session.
                    sim = -1
                item_similalities.append(np.float32(sim))
            df_by_locale = df_by_locale.with_columns(pl.Series(name=f"i2v_similarity_last{last_n}", values=item_similalities).cast(pl.Float32))
            dfs.append(df_by_locale)
        df = pl.concat(dfs)

        # prone similarity between last items and candidates
        dfs = []
        for locale in LOCALES:
            df_by_locale = df.filter(pl.col("locale") == locale)
            last_items = df_by_locale[f"last_item_{last_n}"].to_list()
            cand_items = df_by_locale["candidate_item"].to_list()
            item_similalities = []
            item_index2vector = dict(enumerate(graph_embs[locale]))
            for last_item, cand_item in zip(last_items, cand_items):
                try:
                    item_index1 = item_id2indices_prone[locale][last_item]
                    item_index2 = item_id2indices_prone[locale][cand_item]
                    v1, v2 = item_index2vector[item_index1], item_index2vector[item_index2]
                    sim = calc_cos_sim(v1, v2)
                except (KeyError, TypeError): # KeyError if the item is not in the item2vec training data. TypeError if there are no candidates in a session.
                    sim = -1
                item_similalities.append(np.float32(sim))
            df_by_locale = df_by_locale.with_columns(pl.Series(name=f"graph_emb_similarity_last{last_n}", values=item_similalities).cast(pl.Float32))
            dfs.append(df_by_locale)
        df = pl.concat(dfs)

    return df

def fill_null_and_cast(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns([
        pl.col("co_visit_weight_last1").fill_null(0).cast(pl.Float32),
        pl.col("consective_1_weight_last1").fill_null(0).cast(pl.UInt16),
        pl.col("consective_3_weight_last1").fill_null(0).cast(pl.UInt16),
        pl.col("consective_5_weight_last1").fill_null(0).cast(pl.UInt16),
        pl.col("lift_last1").fill_null(0).cast(pl.Float32),
        pl.col("similarity_score_last1").fill_null(0).cast(pl.Float32),
        pl.col("prone_distance_last1").fill_null(-1).cast(pl.Float32),
        pl.col("co_visit_weight_last2").fill_null(0).cast(pl.Float32),
        pl.col("consective_1_weight_last2").fill_null(0).cast(pl.UInt16),
        pl.col("consective_3_weight_last2").fill_null(0).cast(pl.UInt16),
        pl.col("consective_5_weight_last2").fill_null(0).cast(pl.UInt16),
        pl.col("lift_last2").fill_null(0).cast(pl.Float32),
        pl.col("similarity_score_last2").fill_null(0).cast(pl.Float32),
        pl.col("prone_distance_last2").fill_null(-1).cast(pl.Float32),
        pl.col("co_visit_weight_last3").fill_null(0).cast(pl.Float32),
        pl.col("consective_1_weight_last3").fill_null(0).cast(pl.UInt16),
        pl.col("consective_3_weight_last3").fill_null(0).cast(pl.UInt16),
        pl.col("consective_5_weight_last3").fill_null(0).cast(pl.UInt16),
        pl.col("lift_last3").fill_null(0).cast(pl.Float32),
        pl.col("similarity_score_last3").fill_null(0).cast(pl.Float32),
        pl.col("prone_distance_last3").fill_null(-1).cast(pl.Float32),
        pl.col("imf_score").fill_null(0).cast(pl.Float32),
        pl.col("imf_score_to_recent_intereaction_n3").fill_null(0).cast(pl.Float32),
        pl.col("imf_score_to_recent_intereaction_n5").fill_null(0).cast(pl.Float32),
        pl.col("imf_score_to_recent_intereaction_n2").fill_null(0).cast(pl.Float32),
        pl.col("co_visit_rank_last1").fill_null(999).cast(pl.UInt16),
        pl.col("consective_1_rank_last1").fill_null(999).cast(pl.UInt16),
        pl.col("consective_3_rank_last1").fill_null(999).cast(pl.UInt16),
        pl.col("consective_5_rank_last1").fill_null(999).cast(pl.UInt16),
        pl.col("lift_rank_last1").fill_null(999).cast(pl.UInt16),
        pl.col("similarity_rank_last1").fill_null(999).cast(pl.UInt16),
        pl.col("prone_rank_last1").fill_null(999).cast(pl.UInt16),
        pl.col("co_visit_rank_last2").fill_null(999).cast(pl.UInt16),
        pl.col("consective_1_rank_last2").fill_null(999).cast(pl.UInt16),
        pl.col("consective_3_rank_last2").fill_null(999).cast(pl.UInt16),
        pl.col("consective_5_rank_last2").fill_null(999).cast(pl.UInt16),
        pl.col("lift_rank_last2").fill_null(999).cast(pl.UInt16),
        pl.col("similarity_rank_last2").fill_null(999).cast(pl.UInt16),
        pl.col("prone_rank_last2").fill_null(999).cast(pl.UInt16),
        pl.col("co_visit_rank_last3").fill_null(999).cast(pl.UInt16),
        pl.col("consective_1_rank_last3").fill_null(999).cast(pl.UInt16),
        pl.col("consective_3_rank_last3").fill_null(999).cast(pl.UInt16),
        pl.col("consective_5_rank_last3").fill_null(999).cast(pl.UInt16),
        pl.col("lift_rank_last3").fill_null(999).cast(pl.UInt16),
        pl.col("similarity_rank_last3").fill_null(999).cast(pl.UInt16),
        pl.col("prone_rank_last3").fill_null(999).cast(pl.UInt16),
        pl.col("imf_rank").fill_null(999).cast(pl.UInt16),
        pl.col("imf_rank_to_recent_intereaction_n3").fill_null(999).cast(pl.UInt16),
        pl.col("imf_rank_to_recent_intereaction_n5").fill_null(999).cast(pl.UInt16),
        pl.col("imf_rank_to_recent_intereaction_n2").fill_null(999).cast(pl.UInt16),
        pl.col("S_locale").fill_null(0).cast(pl.UInt8),
        pl.col("S_session_length").fill_null(0).cast(pl.UInt16),
        pl.col("S_nunique_item").fill_null(0).cast(pl.UInt16),
        pl.col("S_nunique_brand").fill_null(0).cast(pl.UInt16),
        pl.col("S_color_not_null_count").fill_null(0).cast(pl.UInt16),
        pl.col("S_size_not_null_count").fill_null(0).cast(pl.UInt16),
        pl.col("S_model_not_null_count").fill_null(0).cast(pl.UInt16),
        pl.col("S_material_not_null_count").fill_null(0).cast(pl.UInt16),
        pl.col("S_author_not_null_count").fill_null(0).cast(pl.UInt16),
        pl.col("S_ratio_unique_brand").fill_null(0).cast(pl.Float32),
        pl.col("S_ratio_repurchase").fill_null(0).cast(pl.Float32),
        # pl.col("S_mean_price").fill_null(0).cast(pl.Float32),
        # pl.col("S_max_price").fill_null(0).cast(pl.Float32),
        # pl.col("S_min_price").fill_null(0).cast(pl.Float32),
        # pl.col("S_std_price").fill_null(0).cast(pl.Float32),
        # pl.col("S_last_item_price").fill_null(0).cast(pl.Float32),
        # pl.col("S_total_amount").fill_null(0).cast(pl.Float32),
        # pl.col("P_price").fill_null(0).cast(pl.Float32),
        pl.col("P_purchase_count").fill_null(0).cast(pl.UInt32),
        pl.col("P_purchase_count_global").fill_null(0).cast(pl.UInt32),
        pl.col("P_n_unique_locale").fill_null(0).cast(pl.UInt8),
        pl.col("P_is_color_null").fill_null(0).cast(pl.UInt8),
        pl.col("P_is_size_null").fill_null(0).cast(pl.UInt8),
        pl.col("P_is_model_null").fill_null(0).cast(pl.UInt8),
        pl.col("P_is_material_null").fill_null(0).cast(pl.UInt8),
        pl.col("P_is_author_null").fill_null(0).cast(pl.UInt8),
        pl.col("P_brand_purchase_count").fill_null(0).cast(pl.UInt16),
        pl.col("P_brand_purchase_count_global").fill_null(0).cast(pl.UInt32),
        # pl.col("P_total_amount").fill_null(0).cast(pl.Float32),
        # pl.col("P_brand_mean_price").fill_null(0).cast(pl.Float32),
        # pl.col("P_brand_max_price").fill_null(0).cast(pl.Float32),
        # pl.col("P_brand_min_price").fill_null(0).cast(pl.Float32),
        # pl.col("P_brand_std_price").fill_null(0).cast(pl.Float32),
        # pl.col("P_total_brand_amount").fill_null(0).cast(pl.Float32),
        pl.col("P_locale_purchase_count").fill_null(0).cast(pl.UInt32),
        # pl.col("P_total_locale_amount").fill_null(0).cast(pl.Float32),
        # pl.col("P_purchase_count_ratio_to_locale").fill_null(0).cast(pl.Float32),
        # pl.col("P_purchase_amount_ratio_to_locale").fill_null(0).cast(pl.Float32),
        # pl.col("P_purchase_count_ratio_to_brand").fill_null(0).cast(pl.Float32),
        # pl.col("P_purchase_amount_ratio_to_brand").fill_null(0).cast(pl.Float32),
        # pl.col("P_price_diff_to_avg_brand_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_price_diff_to_mean_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_price_diff_to_min_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_price_diff_to_max_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_price_diff_to_last_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_brand_price_diff_to_mean_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_brand_price_diff_to_min_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_brand_price_diff_to_max_price").fill_null(0).cast(pl.Float32),
        # pl.col("SP_brand_price_diff_to_last_price").fill_null(0).cast(pl.Float32),
        pl.col("SP_same_brand_last1").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_brand_last2").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_brand_last3").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_color_last1").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_color_last2").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_color_last3").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_size_last1").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_size_last2").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_size_last3").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_model_last1").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_model_last2").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_model_last3").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_material_last1").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_material_last2").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_material_last3").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_author_last1").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_author_last2").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_author_last3").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_brand_sum").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_color_sum").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_size_sum").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_model_sum").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_material_sum").fill_null(0).cast(pl.UInt8),
        pl.col("SP_same_author_sum").fill_null(0).cast(pl.UInt8),
    ])
    return df

## fix seed

In [14]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(SEED)

# Process data

## train

In [15]:
if MAKE_TRAIN:
    n_rows = 50_000
    for idx, df in tqdm(enumerate(train.iter_slices(n_rows=n_rows)), total=math.ceil(train.height/n_rows)): # specify "total" parameter to display tqdm progress bar 
        df = generate_candidates(df, candidate_matrices_train)
        df = df.drop("prev_items")
        df = add_label(df)
        df = filter_null(df, candidate_matrices_train)
        df = filter_session_not_include_positive(df)
        df = negative_sample(df)
        df = add_features(df, session_feat, product_feat_train, i2v_models_train, imf_model_train, user_id2index_train, item_id2index_train, graph_embs_train, item_id2indices_prone_train)
        df = fill_null_and_cast(df)
        df = df[["session_id", "candidate_item", "label"] + USE_FEATURES]
        df.write_parquet(DIR + f"data/interim/for_ranker/task2/train_chunk_{EXP_NAME}_{idx}")

## test

In [16]:
if MAKE_TEST:
    n_rows = 10_000
    for idx, df in tqdm(enumerate(test.iter_slices(n_rows=n_rows)), total=math.ceil(test.height/n_rows)): # specify "total" parameter to display tqdm progress bar 
        # process data
        df = generate_candidates(df, candidate_matrices_test)
        df = df.drop("prev_items")
        df = add_features(df, session_feat, product_feat_test, i2v_models_test, imf_model_test, user_id2index_test, item_id2index_test, graph_embs_test, item_id2indices_prone_test)
        df = fill_null_and_cast(df)
        df.write_parquet(DIR + f"data/interim/for_ranker/task2/test_chunk_{EXP_NAME}_{idx}")

  0%|          | 0/4 [00:00<?, ?it/s]<ipython-input-13-23e2999e636c>:96: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
100%|██████████| 4/4 [54:20<00:00, 815.05s/it]
